In [1]:
import numpy as np
import pandas as pd
import pickle
import requests
import tensorflow as tf
from transformers import TFAutoModel

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
# url='https://kkb-production.jupyter-proxy.kaggle.net/k/143879513/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2IiwidHlwIjoiSldUIn0..4e1hpFVztsBr2Lh2DN8Asw.tantYTVJds0X1r31d_Kc6X4NUR_l9BMTjwHanIY4Dx2BUlrPtGpKQAlnwN70MO2NWivvUjs-cxAU_xMVMyQll_w2Cc9afvf_fkPmr7hVaAK6SkZz405feXMyGar3h1SpWyaRlHmK3RiMCJ8jQqLmNApNGqvX-bArBt18-VITUjF1kxNBMTtf61AKJCSEc1vPpD9RBfrp4qT1yDTAi5VS3w.dU8LAQdvH3RYksVP5DbpQQ/proxy/files/BiomedNLP-PubMedBERT-base-uncased-abstract-ts_inputs.pkl?_xsrf=2%7C41122b45%7C7e453d9ebd4c16961b05a8c35be21bf4%7C1693419082'
# r = requests.get(url, allow_redirects=True)

# open('x_test.zip', 'wb').write(r.content)

167682375

In [ ]:
# with open('', 'wb') as f:
#     pickle.dump(tr_inputs, f)

In [2]:
with open('/kaggle/working/x_test.zip', 'rb') as f:
    x_test = pickle.load(f)
# with open('/kaggle/working/x_train.zip', 'rb') as f:
#     x_tr = pickle.load(f)
# with open('/kaggle/working/x_val.zip', 'rb') as f:
#     x_val = pickle.load(f)

In [3]:
with open('/kaggle/input/y-train-test-val/y_test/kaggle/working/y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)
# with open('/kaggle/input/y-train-test-val/y_tr/kaggle/working/y_tr.pkl', 'rb') as f:
#     y_tr = pickle.load(f)
# with open('/kaggle/input/y-train-test-val/y_val/kaggle/working/y_val.pkl', 'rb') as f:
#     y_val = pickle.load(f)

In [35]:
tr_ds = tf.data.Dataset.from_tensor_slices((x_tr['input_ids'], x_tr['attention_mask'], y_tr))
val_ds = tf.data.Dataset.from_tensor_slices((x_val['input_ids'], x_val['attention_mask'], y_val))

In [36]:
def map_func(input_ids, masks, labels):
    # we convert our three-item tuple into a two-item tuple where the input item is a dictionary
    return {'input_ids': input_ids, 'attention_mask': masks}, labels

# then we use the dataset map method to apply this transformation
tr_ds = tr_ds.map(map_func)
val_ds = val_ds.map(map_func)

In [37]:
BATCH_SIZE = 64
tr_ds = tr_ds.batch(BATCH_SIZE, drop_remainder=True)
val_ds = val_ds.batch(BATCH_SIZE, drop_remainder=True)

In [9]:
bert = TFAutoModel.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract",from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint

In [7]:
N_EPOCHS = 4
LR = 0.001
DECAY = 1e-06
NUM_LABELS = 10

In [15]:
# two input layers, we ensure layer name variables match to dictionary keys in TF dataset
input_ids = tf.keras.layers.Input(shape=(512,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(512,), name='attention_mask', dtype='int32')
bertmodel = bert.bert(input_ids, attention_mask=mask)[1]
drp = tf.keras.layers.Dropout(0.1)(bertmodel)
ln1 = tf.keras.layers.Dense(32, activation='linear')(drp)
bn1 = tf.keras.layers.BatchNormalization()(ln1)
rl1 = tf.keras.layers.Dense(64, activation='relu')(bn1)
ln2 = tf.keras.layers.Dense(32, activation='linear')(rl1)
bn1 = tf.keras.layers.BatchNormalization()(ln2)
rl2 = tf.keras.layers.Dense(64, activation='relu')(bn1)
y = tf.keras.layers.Dense(NUM_LABELS, activation='sigmoid', name='outputs')(rl2)

In [ ]:
# initialize model
model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

# freeze bert layer
model.layers[2].trainable = False

model.summary()

In [ ]:
optimizer = tf.keras.optimizers.AdamW(learning_rate=LR,weight_decay=DECAY)

In [ ]:
loss = tf.keras.losses.BinaryCrossentropy()
metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.Accuracy(),tf.keras.metrics.CategoricalAccuracy()]

In [ ]:
model.compile(optimizer=optimizer, loss=loss,metrics=metrics)

In [ ]:
history = model.fit(
    tr_ds,
    validation_data=val_ds,
    epochs=N_EPOCHS
)

In [ ]:
model.save('BiomedNLP-PubMedBERT-base-uncased-abstract-4')

In [18]:
with open('/kaggle/working/BiomedNLP-PubMedBERT-base-uncased-abstract-4-pkl', 'rb') as f:
    model = pickle.load(f,custom_objects={"TFBertModel": transformers.TFBertModel})

TypeError: 'custom_objects' is an invalid keyword argument for load()

In [21]:
model = load_model('/kaggle/working/BiomedNLP-PubMedBERT-base-uncased-abstract-4',custom_objects={"TFBertModel": TFBertModel})

In [22]:
history = model.fit(
    tr_ds,
    validation_data=val_ds,
    epochs=N_EPOCHS
)

Epoch 1/4
1151/1151 [==============================] - 1838s 2s/step - loss: 0.5339 - precision: 0.5543 - recall: 0.0978 - accuracy: 0.0000e+00 - categorical_accuracy: 0.3843 - val_loss: 0.5229 - val_precision: 0.5788 - val_recall: 0.1109 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.4133
Epoch 2/4
1151/1151 [==============================] - 1865s 2s/step - loss: 0.5308 - precision: 0.5577 - recall: 0.1105 - accuracy: 0.0000e+00 - categorical_accuracy: 0.3811 - val_loss: 0.5202 - val_precision: 0.5666 - val_recall: 0.1468 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.3612
Epoch 3/4
1151/1151 [==============================] - 1865s 2s/step - loss: 0.5285 - precision: 0.5646 - recall: 0.1191 - accuracy: 0.0000e+00 - categorical_accuracy: 0.3782 - val_loss: 0.5190 - val_precision: 0.5747 - val_recall: 0.1547 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.3878


In [23]:
model.save('BiomedNLP-PubMedBERT-base-uncased-abstract-8')

In [24]:
with open('BiomedNLP-PubMedBERT-base-uncased-abstract-8-pkl', 'wb') as f:
    pickle.dump(model, f)

/opt/conda/lib/python3.10/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [25]:
history = model.fit(
    tr_ds,
    validation_data=val_ds,
    epochs=N_EPOCHS
)

Epoch 1/4
1151/1151 [==============================] - 1825s 2s/step - loss: 0.5275 - precision: 0.5659 - recall: 0.1254 - accuracy: 0.0000e+00 - categorical_accuracy: 0.3792 - val_loss: 0.5158 - val_precision: 0.6051 - val_recall: 0.1046 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.3893
Epoch 2/4
1151/1151 [==============================] - 1865s 2s/step - loss: 0.5263 - precision: 0.5631 - recall: 0.1303 - accuracy: 0.0000e+00 - categorical_accuracy: 0.3773 - val_loss: 0.5133 - val_precision: 0.5877 - val_recall: 0.1411 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.3857


In [28]:
model.save('BiomedNLP-PubMedBERT-base-uncased-abstract-12')

In [29]:
with open('BiomedNLP-PubMedBERT-base-uncased-abstract-12-pkl', 'wb') as f:
    pickle.dump(model, f)

/opt/conda/lib/python3.10/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [38]:
history = model.fit(
    tr_ds,
    validation_data=val_ds,
    epochs=4
)

Epoch 1/4
1151/1151 [==============================] - 1840s 2s/step - loss: 0.5263 - precision: 0.5650 - recall: 0.1319 - accuracy: 0.0000e+00 - categorical_accuracy: 0.3742 - val_loss: 0.5145 - val_precision: 0.6000 - val_recall: 0.1129 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.4110
Epoch 2/4
1151/1151 [==============================] - 1865s 2s/step - loss: 0.5257 - precision: 0.5642 - recall: 0.1343 - accuracy: 0.0000e+00 - categorical_accuracy: 0.3784 - val_loss: 0.5156 - val_precision: 0.6009 - val_recall: 0.1245 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.3921
Epoch 4/4
1151/1151 [==============================] - 1825s 2s/step - loss: 0.5255 - precision: 0.5680 - recall: 0.1337 - accuracy: 0.0000e+00 - categorical_accuracy: 0.3771 - val_loss: 0.5137 - val_precision: 0.5904 - val_recall: 0.1409 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.3994


In [56]:
print(history.history['loss'])
print(history.history['val_loss'])

[0.5262900590896606, 0.5258864760398865, 0.5257043242454529, 0.5254917144775391]
[0.5144523978233337, 0.5148446559906006, 0.5156236290931702, 0.5136582255363464]


In [39]:
model.save('BiomedNLP-PubMedBERT-base-uncased-abstract-13')
with open('BiomedNLP-PubMedBERT-base-uncased-abstract-13-pkl', 'wb') as f:
    pickle.dump(model, f)

/opt/conda/lib/python3.10/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


Prediction

In [6]:
threshold  = np.arange(0.1,0.9,0.01)
threshold

array([0.1 , 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 ,
       0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31,
       0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42,
       0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53,
       0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64,
       0.65, 0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75,
       0.76, 0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86,
       0.87, 0.88, 0.89])

In [4]:
from tensorflow.keras.models import load_model
from transformers import TFBertModel
#model_1 = load_model('BiomedNLP-PubMedBERT-base-uncased-abstract-4',custom_objects={"TFBertModel": TFBertModel})
#model_2 = load_model('BiomedNLP-PubMedBERT-base-uncased-abstract-8',custom_objects={"TFBertModel": TFBertModel})
#model_3 = load_model('BiomedNLP-PubMedBERT-base-uncased-abstract-12',custom_objects={"TFBertModel": TFBertModel})
model_4 = load_model('BiomedNLP-PubMedBERT-base-uncased-abstract-13',custom_objects={"TFBertModel": TFBertModel}) # 16 epoch

In [5]:
#probes_1 = model_1.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
#probes_2 = model_2.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
#probes_3 = model_3.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
probes_4 = model_4.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})

640/640 [==============================] - 433s 673ms/step


In [11]:
with open('13-predictions', 'wb') as f:
    pickle.dump(probes_4, f)

In [20]:
y_test

array([[1, 0, 0, ..., 0, 1, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 1],
       [1, 1, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 1, 0]])

In [32]:
import numpy as np
from sklearn.metrics import roc_curve, precision_score, recall_score, f1_score

# Example predicted labels (probabilities)
predicted_labels = probes_4
# True labels
true_labels = y_test

# Determine threshold for each label
thresholds = []
for label_id in range(predicted_labels.shape[1]):
    # Get probabilities for the current label
    label_probs = predicted_labels[:, label_id]
    # Get true labels for the current label
    label_true = true_labels[:, label_id]
    # Calculate the false positive rate, true positive rate, and thresholds
    fpr, tpr, roc_thresholds = roc_curve(label_true, label_probs)
    # Calculate the distances from the top-left corner (0, 1)
    distances = np.sqrt(np.square(1 - tpr) + np.square(fpr))
    # Find the index of the threshold that minimizes the distance
    best_index = np.argmin(distances)
    # Get the corresponding threshold
    best_threshold = roc_thresholds[best_index]
    thresholds.append(best_threshold)
# Print the thresholds for each label
for label_id, threshold in enumerate(thresholds):
    print(f"Label {label_id + 1}: {threshold}")

Label 1: 0.4252188503742218
Label 2: 0.43179163336753845
Label 3: 0.36237621307373047
Label 4: 0.27204763889312744
Label 5: 0.2108398675918579
Label 6: 0.22177961468696594
Label 7: 0.1640295386314392
Label 8: 0.24510458111763
Label 9: 0.14194940030574799
Label 10: 0.1622004508972168


In [27]:
binary_predictions = (predicted_labels >= thresholds).astype(int)

In [28]:
cols = ['I10', 'E785', 'Z87891', 'K219', 'F329', 'I2510', 'F419', 'N179',
       'Z794', 'E039']

In [31]:
print("Classification_report:")
print(metrics.classification_report(y_test,binary_predictions,target_names=cols,zero_division=0))

Classification_report:
              precision    recall  f1-score   support

         I10       0.53      0.62      0.57      8790
        E785       0.55      0.63      0.59      8848
      Z87891       0.45      0.58      0.51      7418
        K219       0.38      0.57      0.46      6204
        F329       0.31      0.60      0.40      4676
       I2510       0.44      0.68      0.54      4595
        F419       0.27      0.60      0.37      3859
        N179       0.31      0.69      0.43      3932
        Z794       0.26      0.68      0.37      3139
        E039       0.22      0.61      0.32      3018

   micro avg       0.38      0.62      0.47     54479
   macro avg       0.37      0.63      0.46     54479
weighted avg       0.41      0.62      0.49     54479
 samples avg       0.39      0.62      0.44     54479

